### Q#1 Given the Reuters-21578 dataset, please calculate the term frequencies, and output the representation of the document contents as a term-document count matrix.

In [1]:
import re
# function to clear characters that are not necessary
def removeTrashChar(string, trash):
    for i in trash:
        string = string.replace(i, ' ')
    return re.sub(' +', ' ', string)

In [2]:
#data split and reduce
data = sc.wholeTextFiles('dataset/reuters/reut2-*')
news_articles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0])\
                    .map(lambda x:removeTrashChar(x, '~!@#$%^&*()_+`-=/\\,.<>\'\";:{}[]\n\t|'))
print("There are {} News Articls".format(news_articles.count()))

There are 19043 News Articls


In [3]:
#select distinict words from the filtered article above
distinct_words = news_articles.flatMap(lambda x:x.split(' ')).distinct() 
print("There are {} distinct words".format(distinct_words.count()))

There are 48447 distinct words


In [4]:
from pyspark.mllib.linalg.distributed import RowMatrix
news_articles_list = news_articles.map(lambda x:x.split(' ')).collect()
term_document_count_matrix = RowMatrix(distinct_words.map(lambda x:[article.count(x) for article in news_articles_list]))

In [5]:
term_document_count_matrix.rows.coalesce(1).saveAsTextFile('tmp')

In [7]:
!mv tmp/part-00000 result/term_document_count_matrix.txt

### Both Q#2 & Q#3
(2) Implement matrix multiplication by MapReduce. Your program  should be able to output the result in appropriate dimensions.
<br>(3) Given the term-document matrix in (1), compute the SVD decomposition of the matrix using MapReduce. Output the resulting eigenvalues and eigenvectors. 

In [ ]:
R = 10
svd = term_document_count_matrix.computeSVD(R)
matrixMultiply = term_document_count_matrix.multiply(svd.V)

In [ ]:
matrixMultiply.rows.coalesce(1).saveAsTextFile('temp_q2')
!mv temp_q2/part-00000 result/matrixMultiply.txt
#!rm -rf tmp

In [ ]:
V = svd.V.toArray()
s = svd.s.toArray()
with open('result/svd.txt', 'w') as result:
    for i in range(R):
        result.write(str(s[i]) + '\n' + str(V[i]) + '\n\n')